In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch
import pickle


In [3]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [41]:
sats_games = []
PREFIXES = ['jun26', 'aug31']
for prefix in PREFIXES:
    sats_games += [f'{prefix}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{prefix}/*')]
    
done = set([s for s in sats_games if ('base' in s or 'speed' in s) and 'spite' not in s and 'risk' not in s] \
    + [s for s in sats_games if ('activity' in s) or ('grace' in s and not 'spite' in s)])
sats_games = list(set(sats_games) - done)
print(sats_games)

['jun26/jun26_3_spite_grace', 'jun26/jun26_4_risk_averse_tie_break', 'aug31/aug31_0_hide_demand', 'jun26/jun26_0_hide_demand', 'jun26/jun26_2_risk_averse', 'aug31/aug31_4_undersell_allowed', 'aug31/aug31_3_tie_break', 'jun26/jun26_4_hide_demand', 'jun26/jun26_3_spite', 'aug31/aug31_2_undersell_allowed', 'jun26/jun26_3_undersell_allowed', 'jun26/jun26_1_spite', 'aug31/aug31_4_risk_averse_tie_break', 'aug31/aug31_2_risk_averse_tie_break', 'jun26/jun26_2_tie_break', 'jun26/jun26_0_undersell_allowed', 'jun26/jun26_4_tie_break', 'jun26/jun26_1_tie_break', 'jun26/jun26_3_risk_averse_tie_break', 'jun26/jun26_3_tie_break', 'jun26/jun26_0_spite', 'aug31/aug31_0_undersell_allowed', 'jun26/jun26_4_risk_averse', 'jun26/jun26_1_spite_grace', 'aug31/aug31_1_hide_demand', 'aug31/aug31_2_spite', 'aug31/aug31_4_tie_break', 'jun26/jun26_1_risk_averse', 'jun26/jun26_2_undersell_allowed', 'aug31/aug31_3_risk_averse_tie_break', 'jun26/jun26_1_risk_averse_tie_break', 'aug31/aug31_0_spite', 'jun26/jun26_3_hi

# For running the real dumb test case

In [46]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

70


In [ ]:
# num_iters = 999_000_000
# time_limit_seconds = int(3600 * 0.5)
# for config in pgrid:
#     CONFIG = 'cfr_init_2' # The YML network config folder for the main jobs
#     N_SEEDS = 1 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_999_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun24_cfr_gamut_7', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [43]:
num_iters = 999_000_000
time_limit_seconds = int(3600 * 9)
for config in pgrid:
    CONFIG = 'cfr_port_2_no_ext' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 9000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='sep1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 93193
Submitted batch job 93194
Submitted batch job 93195
Submitted batch job 93196
Submitted batch job 93197
Submitted batch job 93198
Submitted batch job 93199
Submitted batch job 93200
Submitted batch job 93201
Submitted batch job 93202
Submitted batch job 93203
Submitted batch job 93204
Submitted batch job 93205
Submitted batch job 93206
Submitted batch job 93207
Submitted batch job 93208
Submitted batch job 93209
Submitted batch job 93210
Submitted batch job 93211
Submitted batch job 93212
Submitted batch job 93213
Submitted batch job 93214
Submitted batch job 93215
Submitted batch job 93216
Submitted batch job 93217
Submitted batch job 93218
Submitted batch job 93219
Submitted batch job 93220
Submitted batch job 93221
Submitted batch job 93222
Submitted batch job 93223
Submitted batch job 93224
Submitted batch job 93225
Submitted batch job 93226
Submitted batch job 93227
Submitted batch job 93228
Submitted batch job 93229
Submitted batch job 93230
Submitted ba

In [22]:
# PREFIX = 'sep1'
# sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
# print(sats_games)
# sats_games = \
#     [s for s in sats_games if ('hide' in s)]
# pgrid = ParameterGrid([
#     {
#         # Defaults
#         'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
#     },
# ])
# list(pgrid)

In [31]:
### Test 3 players

PREFIX = 'sep1'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
# sats_games = [s for s in sats_games if 'hide' not in s and 'no_activity' not in s and 'undersell' not in s and 'speed' not in s and 'spite' not in s]
# sats_games = [s for s in sats_games if ('base' in s or 'grace' in s or 'activity' in s or 'speed' in s) and 'spite' not in s and 'risk' not in s]
sats_games = \
    [s for s in sats_games if ('hide' in s)]
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])

num_iters = 999_000_000
time_limit_seconds = int(3600 * 9)
for config in pgrid:
    CONFIG = 'cfr_port_2_no_ext' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 999_999_999 --eval_every_seconds 9000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='3players_sep1', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 93163
Submitted batch job 93164
Submitted batch job 93165
Submitted batch job 93166
Submitted batch job 93167
Submitted batch job 93168
Submitted batch job 93169
Submitted batch job 93170
Submitted batch job 93171
Submitted batch job 93172
Submitted batch job 93173
Submitted batch job 93174
Submitted batch job 93175
Submitted batch job 93176
Submitted batch job 93177
Submitted batch job 93178
Submitted batch job 93179
Submitted batch job 93180
Submitted batch job 93181
Submitted batch job 93182
Submitted batch job 93183
Submitted batch job 93184
Submitted batch job 93185
Submitted batch job 93186
Submitted batch job 93187
Submitted batch job 93188
Submitted batch job 93189
Submitted batch job 93190
Submitted batch job 93191
Submitted batch job 93192


In [ ]:
# num_iters = 10_000_000
# time_limit_seconds = 3600 * 4
# for config in pgrid:
#     CONFIG = 'cfr_external' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 250_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun23external', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
# num_iters = 30_000_000
# time_limit_seconds = 3600 * 4
# for config in pgrid:
#     CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --eval_every 999_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun23outcome_2', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

In [ ]:
# for config in pgrid:
#     CONFIG = 'ppo_no_ent_jun14'
#     # CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
#     # BR_CONFIG = 'ppo_may31_23' 
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun14ent_3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

In [48]:
!squeue --format="%.18i %.90j %.2t .%.10M%.6D %R"

             JOBID                                                                                       NAME ST .      TIME NODES NODELIST(REASON)


In [ ]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [ ]:
check_queue()

In [106]:
!squeue | wc -l

5


In [ ]:
!squeue | grep "may30" | wc -l

In [29]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")

In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "eval" | awk '{print $1}' | xargs scancel

In [ ]:
from compress_pickle import dumps, loads


In [ ]:
loads?

In [ ]:
from compress_pickle.compressers.registry import get_known_compressions

In [ ]:
get_known_compressions()

In [ ]:
x = EquilibriumSolverRunCheckpoint.objects.last().policy

In [ ]:
len(x)

In [ ]:
len(dumps(pickle.loads(x), compression='bz2'))

In [ ]:
len(pickle.dumps(pickle.loads(x)))

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().walltime

In [ ]:
EquilibriumSolverRunCheckpoint.objects.last().t